# Training

## Prepa des data

In [2]:

import os
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = "/jovyan/data/reservation-first-dataset-train.csv"

# Charger les données
try:
    data = pd.read_csv(file_path)
    data = data.fillna('')  # Pour les valeurs NaN possibles
    print("Fichier chargé avec succès")
except FileNotFoundError:
    print("Le fichier n'a pas été trouvé. Veuillez vérifier le chemin.")




LoadError: ArgumentError: Package os not found in current path.
- Run `import Pkg; Pkg.add("os")` to install the os package.

## Tokenisation

In [15]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Chemin relatif pour accéder au fichier CSV dans un dossier `data` adjacent à `sandbox`
file_path = os.path.join('..', 'data', 'reservation-first-dataset-train.csv')
absolute_file_path = os.path.abspath(file_path)

# Afficher le chemin absolu du fichier CSV
print("Chemin absolu du fichier:", absolute_file_path)

# Charger les données
try:
    data = pd.read_csv(absolute_file_path)
    data = data.fillna('')  # Pour les valeurs NaN possibles
    print("Fichier chargé avec succès")
    print("Aperçu des données :")
    print(data.head())  # Afficher les premières lignes des données pour vérifier leur contenu
    print("Colonnes des données:", data.columns)  # Vérifier les colonnes
except FileNotFoundError:
    print("Le fichier n'a pas été trouvé. Veuillez vérifier le chemin.")
    data = None

# Assurez-vous que data est bien défini avant de continuer
if data is not None:
    sentences = []
    tags = []

    for idx, row in data.iterrows():
        phrase = row['Phrase']
        depart = row['Départ']
        arrivee = row['Arrivée']

        words = phrase.split()
        tag_sequence = []

        for word in words:
            if word in depart.split():
                tag_sequence.append("B-Départ")
            elif word in arrivee.split():
                tag_sequence.append("B-Arrivée")
            else:
                tag_sequence.append("O")

        sentences.append(words)
        tags.append(tag_sequence)

    # Split data into train and test sets
    train_sentences, val_sentences, train_tags, val_tags = train_test_split(sentences, tags, test_size=0.1)

    print("Nombre de phrases d'entraînement:", len(train_sentences))
    print("Nombre de phrases de validation:", len(val_sentences))
else:
    print("Impossible de traiter les données car elles n'ont pas été chargées.")


Chemin absolu du fichier: /home/jovyan/model/SANDRINE/data/reservation-first-dataset-train.csv
Le fichier n'a pas été trouvé. Veuillez vérifier le chemin.
Impossible de traiter les données car elles n'ont pas été chargées.


## Construction et Entraînement du Modèle TensorFlow

In [ ]:

from transformers import TFAutoModelForTokenClassification
from transformers import create_optimizer
import tensorflow_addons as tfa

# Charger un modèle pré-entraîné adapté pour la classification des tokens
model = TFAutoModelForTokenClassification.from_pretrained('camembert-base', num_labels=len(label_list))

# Optimizer et Schedule de l'apprentissage
batch_size = 8
num_train_steps = len(train_tokenized["input_ids"]) // batch_size * 3  # 3 epochs
num_warmup_steps = num_train_steps // 10

optimizer, schedule = create_optimizer(init_lr=2e-5, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

# Préparer les datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_tokenized), train_tokenized["labels"]))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_tokenized), val_tokenized["labels"]))

train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Entraîner le modèle
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)


# Évaluation du Modèle

In [ ]:

import numpy as np

# Prédire sur le jeu de validation
predictions = model.predict(val_dataset)
predictions = np.argmax(predictions.logits, axis=-1)

true_labels = [[label_list[label] for label in labels if label != -100] for labels in val_tokenized["labels"]]
predicted_labels = [[label_list[p] for (p, l) in zip(prediction, labels) if l != -100] for prediction, labels in zip(predictions, val_tokenized["labels"])]

# Afficher un rapport de classification
from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_labels))
